In [ ]:
!pip install --upgrade --quiet pip
!pip install --quiet python-dotenv
!pip install --quiet langchain langgraph langchain_community
!pip install --quiet transformers mistralai langchain_mistralai

In [2]:
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

In [18]:
from langgraph.graph import StateGraph, END
from typing import TypedDict, Annotated
from langchain_core.messages import AnyMessage, SystemMessage, HumanMessage, ToolMessage
from langchain_community.tools.tavily_search import TavilySearchResults
from mistralai import Mistral
from langchain_mistralai.chat_models import ChatMistralAI

import operator
import os

In [9]:
tool = TavilySearchResults(max_results=2)
print(type(tool))
print(tool.name)

<class 'langchain_community.tools.tavily_search.tool.TavilySearchResults'>
tavily_search_results_json


In [4]:
class AgentState(TypedDict):
  messages: Annotated[list[AnyMessage], operator.add]

In [30]:
class Agent:
  def __init__(self, model, tools, system=""):
    self.system = system
    graph = StateGraph(AgentState)
    graph.add_node("llm", self.call_mistral)
    graph.add_node("action", self.take_action)
    graph.add_conditional_edges(
        "llm",
        self.exists_action,
        {True: "action", False: END})
    graph.add_edge("action", "llm")
    graph.set_entry_point("llm")
    self.graph = graph.compile()
    self.tools = {tool.name: tool for tool in tools}
    self.model = model.bind_tools(tools)

  def call_mistral(self, state: AgentState):
    messages = state['messages']
    if self.system:
      messages = [SystemMessage(content=self.system)] + messages
    message = self.model.invoke(messages)
    return {'messages': [message]}

  def take_action(self, state: AgentState):
    tool_calls = state['messages'][-1].tool_calls
    results = []
    for t in tool_calls:
      print(f"Calling: {t}")
      result = self.tools[t['name']].invoke(t['args'])
      results.append(ToolMessage(tool_call_id=t['id'], name=t['name'], content=str(result)))
    print("Back to model!")
    return {'messages': results}

  def exists_action(self, state: AgentState):
    return len(state['messages'][-1].tool_calls) > 0


In [19]:
# save MISTRAL_API_KEY in .env file
api_key = os.getenv("MISTRAL_API_KEY")
model = ChatMistralAI(model="mistral-large-latest", api_key=api_key)

In [31]:
prompt = """You are a smart research assistant. Use the search engine to look up information. \
You are allowed to make multiple calls (either together or in sequence). \
Only look up information when you are sure of what you want. \
If you need to look up some information before asking a follow up question, you are allowed to do that!
"""
abot = Agent(model, [tool], system=prompt)

In [32]:
messages = [HumanMessage(content="What is the weather in sf?")]
result = abot.graph.invoke({"messages": messages})

Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'weather in sf'}, 'id': 'hvwiJspLY', 'type': 'tool_call'}
Back to model!


In [33]:
result

{'messages': [HumanMessage(content='What is the weather in sf?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'hvwiJspLY', 'function': {'name': 'tavily_search_results_json', 'arguments': '{"query": "weather in sf"}'}, 'index': 0}]}, response_metadata={'token_usage': {'prompt_tokens': 182, 'total_tokens': 212, 'completion_tokens': 30}, 'model': 'mistral-large-latest', 'finish_reason': 'tool_calls'}, id='run-b7765b57-f24a-46da-8de5-fa4e5566ef61-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'weather in sf'}, 'id': 'hvwiJspLY', 'type': 'tool_call'}], usage_metadata={'input_tokens': 182, 'output_tokens': 30, 'total_tokens': 212}),
  ToolMessage(content='[{\'url\': \'https://www.weatherapi.com/\', \'content\': "{\'location\': {\'name\': \'San Francisco\', \'region\': \'California\', \'country\': \'United States of America\', \'lat\': 37.775, \'lon\': -122.4183, \'tz_id\': \'America/Los_Angeles\', \'lo

In [34]:
result['messages'][-1].content

'The current weather in San Francisco is overcast with a temperature of 16.1°C (61.0°F). The wind is blowing from the south at 21.9 mph (35.3 km/h), with gusts up to 33.4 mph (53.7 km/h). The humidity is 67%, and the pressure is 1010.0 mb (29.81 in). There is no precipitation at the moment. The visibility is 16.0 km (9.0 miles), and the UV index is 0.0. The dew point is 11.1°C (51.9°F), and the heat index is 12.6°C (54.7°F). The wind chill is 10.4°C (50.6°F). The last update was at 2025-02-03 18:00 local time.\n\nFor April 2025, the weather in San Francisco is expected to be mild, with temperatures ranging from 50°F to 64°F. The average high temperature will be around 61°F, and the average low temperature will be around 52°F. The weather is expected to be mostly clear, with some cloudy days and a few days with light rain. The average humidity will be around 70%, and the average wind speed will be around 10 mph. The weather is expected to be similar to previous years, with no major weat

In [35]:
messages = [HumanMessage(content="What is the weather in SF and LA?")]
result = abot.graph.invoke({"messages": messages})

Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'What is the weather in SF?'}, 'id': 'fi8YMxBcu', 'type': 'tool_call'}
Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'What is the weather in LA?'}, 'id': 'MVVOvJJuz', 'type': 'tool_call'}
Back to model!


In [36]:
result['messages'][-1].content

'The weather in San Francisco is overcast with a temperature of 61.0°F (16.1°C) and wind speeds of 21.7 mph (34.9 km/h). The weather in Los Angeles is clear with a temperature of 64.4°F (18.0°C) and wind speeds of 7.8 mph (12.6 km/h).'

In [37]:
# Note, the query was modified to produce more consistent results.
# Results may vary per run and over time as search information and models change.

query = "Who won the super bowl in 2024? In what state is the winning team headquarters located? \
What is the GDP of that state? Answer each question."
messages = [HumanMessage(content=query)]

result = abot.graph.invoke({"messages": messages})

Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'Who won the super bowl in 2024'}, 'id': 'sgK4LQFfe', 'type': 'tool_call'}
Back to model!
Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'In what state is the Kansas City Chiefs headquarters located?'}, 'id': 'a0AM32eQH', 'type': 'tool_call'}
Back to model!
Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'What is the GDP of Missouri?'}, 'id': 'DJdlaXFL5', 'type': 'tool_call'}
Back to model!


In [38]:
print(result['messages'][-1].content)

**The Kansas City Chiefs won the Super Bowl in 2024**. The Kansas City Chiefs headquarters is located in Kansas City, Missouri. The GDP of Missouri is $344.12 billion USD.
